In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd

# Replace with your actual file path
file_path = '/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/Health_and_Personal_Care.jsonl'

# Load the .jsonl file into a pandas DataFrame
df = pd.read_json(file_path, lines=True)

# Display the first 5 rows of the DataFrame
print("DataFrame Loaded Successfully!")
df.head()


DataFrame Loaded Successfully!


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,[],B07TDSJZMR,B07TDSJZMR,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-02-06 00:49:35.902,3,True
1,5,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,[],B08637FWWF,B08637FWWF,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2020-11-02 22:03:06.880,3,True
2,5,Fantastic,I have been suffering a couple months with hee...,[],B07KJVGNN5,B07KJVGNN5,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,2019-07-24 11:13:58.905,0,True
3,4,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",[],B007HY7GC2,B092RP73CX,AEZGPLOYTSAPR3DHZKKXEFPAXUAA,2022-09-04 02:29:02.725,7,True
4,1,Not for me,Didn't do a thing for me. Not saying they don'...,[],B08KYJLF5T,B08KYJLF5T,AEQAYV7RXZEBXMQIQPL6KCT2CFWQ,2022-01-20 23:53:07.262,0,True


In [ ]:
# number of reviews
print(len(df))

print(list(df.columns))

494121
['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase']


In [ ]:
# count of all ratings to check if there's imbalance
df['rating'].value_counts()

,count
rating,
5,301713
1,69564
4,57000
3,36949
2,28895


There seems to be a class imbalance. 5 star reviews seem to dominate a lot.

In [ ]:
df.isnull().sum()

,0
rating,0
title,0
text,0
images,0
asin,0
parent_asin,0
user_id,0
timestamp,0
helpful_vote,0
verified_purchase,0


There doesn't seem to be any null values.

In [ ]:
!pip install gensim -q # Install gensim for loading embeddings
import gensim
import gensim.downloader as api # To potentially download models easily
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import os # For file path handling
import gzip # For potential .gz files
import requests # For downloading
from tqdm.notebook import tqdm # Progress bar for loops

from google.colab import drive
from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer # No longer needed
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time

# Preprocessing

In [ ]:
df.dropna(subset=['text', 'rating'], inplace=True)
print(f"\nDataFrame shape after dropping rows with missing text/rating: {df.shape}")

# Combine 'title' and 'text'
# Fill missing titles with an empty string BEFORE concatenation
df['title'] = df['title'].fillna('')
df['review_full'] = df['title'] + ' ' + df['text']

# Text Cleaning Setup
nltk.download('punkt_tab', quiet=True) # Download the specific resource needed
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # 1. Lowercasing
    text = text.lower()
    # 2. Remove HTML tags (if any)
    text = re.sub(r'<.*?>', '', text)
    # 3. Remove punctuation and special characters (keeping only letters and whitespace)
    text = re.sub(r'[^a-z\s]', '', text)
    # 4. Tokenization
    tokens = word_tokenize(text)
    # 5. Remove Stop Words and Lemmatize
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 1] # Keep words with length > 1
    # 6. Join back into string
    return ' '.join(cleaned_tokens)

print("\nStarting text cleaning (this may take a while)...")
# Apply cleaning function - Monitor progress if needed for large datasets
# Consider df['review_full'].parallel_apply(clean_text) using libraries like pandarallel if speed is critical
df['review_cleaned'] = df['review_full'].apply(clean_text)
print("Text cleaning completed.")

# Drop rows where cleaning might have resulted in empty strings
df.dropna(subset=['review_cleaned'], inplace=True)
df = df[df['review_cleaned'].str.strip() != ''] # Ensure no empty strings after cleaning
print(f"DataFrame shape after cleaning and removing empty reviews: {df.shape}")


DataFrame shape after dropping rows with missing text/rating: (494121, 11)

Starting text cleaning (this may take a while)...
Text cleaning completed.
DataFrame shape after cleaning and removing empty reviews: (493820, 12)


In [ ]:
df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,review_full,review_cleaned
0,4,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,[],B07TDSJZMR,B07TDSJZMR,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-02-06 00:49:35.902,3,True,12 mg is 12 on the periodic table people! Mg f...,mg periodic table people mg magnesium review c...
1,5,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,[],B08637FWWF,B08637FWWF,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2020-11-02 22:03:06.880,3,True,Save the lanet using less plastic. Love these ...,save lanet using less plastic love easy multit...
2,5,Fantastic,I have been suffering a couple months with hee...,[],B07KJVGNN5,B07KJVGNN5,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,2019-07-24 11:13:58.905,0,True,Fantastic I have been suffering a couple month...,fantastic suffering couple month heel pain pla...
3,4,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",[],B007HY7GC2,B092RP73CX,AEZGPLOYTSAPR3DHZKKXEFPAXUAA,2022-09-04 02:29:02.725,7,True,It holds the water and makes bubbles. That's ...,hold water make bubble thats bought cheap want...
4,1,Not for me,Didn't do a thing for me. Not saying they don'...,[],B08KYJLF5T,B08KYJLF5T,AEQAYV7RXZEBXMQIQPL6KCT2CFWQ,2022-01-20 23:53:07.262,0,True,Not for me Didn't do a thing for me. Not sayin...,didnt thing saying dont


Saving the processed data frame in a file (pickle, parquet and csv) as a checkpoints.

In [ ]:
df.to_pickle('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.pkl')

In [ ]:
df.to_parquet('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.parquet')

In [ ]:
df.to_csv('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.csv', index=False)

In [ ]:
import pandas as pd

# Load the parquet file
df = pd.read_parquet('/content/drive/MyDrive/DADM/Main_Assignment_Shared_resources/Mervin/Main_Assignment/reviews_with_full_column.parquet')

# --- 4. Data Splitting ---

In [ ]:
# Define Features (X) and Target (y)
X = df['review_cleaned']
y = df['rating']
print("Splitting data into Training and Testing sets (80/20)...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 20% for testing
    random_state=42,  # for reproducibility
    stratify=y        # IMPORTANT: keep class distribution same in train/test due to imbalance
)
print(f"Training set size: {len(X_train)} samples")
print(f"Testing set size: {len(X_test)} samples")


Splitting data into Training and Testing sets (80/20)...
Training set size: 395056 samples
Testing set size: 98764 samples


# --- Load FastText Embeddings ---

In [ ]:
# Option 1: Use gensim downloader (might be smaller models)
print("\nAttempting to load pre-trained FastText model via gensim downloader...")
try:
    # Example: fasttext-wiki-news-subwords-300 (smaller, handles OOV better)
    ft_model = api.load("fasttext-wiki-news-subwords-300")
    vector_size = ft_model.vector_size
    print(f"Loaded 'fasttext-wiki-news-subwords-300' with vector size {vector_size}")
except ValueError as e:
    print(f"Could not download model via gensim: {e}")
    print("Proceeding to manual download option...")
    ft_model = None


Attempting to load pre-trained FastText model via gensim downloader...
[==================================================] 100.0% 958.5/958.4MB downloaded
Loaded 'fasttext-wiki-news-subwords-300' with vector size 300


# <<< --- NEW SECTION: Vectorize Reviews using Embeddings --- >>>

In [ ]:
nltk.download('punkt_tab', quiet=True) # Download the specific resource needed
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

True

In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize # Make sure this is imported
from tqdm.notebook import tqdm # Make sure this is imported

def vectorize_text(text, model, vector_size):
    """Calculates the average embedding vector for a text."""
    tokens = word_tokenize(text) # Ensure text is tokenized
    vectors = []
    if model: # Check if model loaded successfully
        for token in tokens:
            # --- CHANGE HERE ---
            # Check directly in the model's vocabulary
            if token in model:
                # Access the vector directly from the model
                vectors.append(model[token])
            # --- END OF CHANGE ---

    if not vectors: # Handle empty reviews or all OOV words
        return np.zeros(vector_size)
    else:
        return np.mean(vectors, axis=0)

# --- The code applying this function remains the same ---
if ft_model:
    print("\nVectorizing training data (Averaging FastText Embeddings)...")
    X_train_embeddings = np.array([vectorize_text(text, ft_model, vector_size) for text in tqdm(X_train, desc="Vectorizing Train")])
    print(f"Training data vectorized. Shape: {X_train_embeddings.shape}")

    print("Vectorizing testing data...")
    X_test_embeddings = np.array([vectorize_text(text, ft_model, vector_size) for text in tqdm(X_test, desc="Vectorizing Test")])
    print(f"Testing data vectorized. Shape: {X_test_embeddings.shape}")
else:
    print("\nSkipping vectorization and training as FastText model failed to load.")
    # Set dummy variables or exit if you prefer
    X_train_embeddings, X_test_embeddings = None, None


Vectorizing training data (Averaging FastText Embeddings)...


Vectorizing Train:   0%|          | 0/395056 [00:00<?, ?it/s]

Training data vectorized. Shape: (395056, 300)
Vectorizing testing data...


Vectorizing Test:   0%|          | 0/98764 [00:00<?, ?it/s]

Testing data vectorized. Shape: (98764, 300)


# --- Model Training (Using Embeddings) ---

In [ ]:
print("\nTraining Logistic Regression model with class_weight='balanced' on averaged embeddings...")
# Using the same Logistic Regression setup
log_reg_model_emb = LogisticRegression(
    class_weight='balanced',
    solver='liblinear', # Still a good choice
    max_iter=2000,      # Keep increased iterations
    random_state=42
)

start_train = time.time()
log_reg_model_emb.fit(X_train_embeddings, y_train) # Fit on embeddings
print(f"Model training completed in {time.time() - start_train:.2f} seconds.")


Training Logistic Regression model with class_weight='balanced' on averaged embeddings...
Model training completed in 303.77 seconds.


# --- 6. Model Evaluation ---

In [ ]:
print("\nEvaluating model performance on the Test set (using embeddings)...")

# Predict on the test set using the model trained on embeddings
y_pred_emb = log_reg_model_emb.predict(X_test_embeddings) # Predict on embeddings

# Calculate Accuracy
accuracy_emb = accuracy_score(y_test, y_pred_emb)
print(f"\nOverall Accuracy (Embeddings): {accuracy_emb:.4f}")

# Detailed Classification Report
print("\nClassification Report (Embeddings):")
print(classification_report(y_test, y_pred_emb, target_names=[str(i) for i in sorted(y.unique())], zero_division=0))

# Confusion Matrix
print("\nConfusion Matrix (Embeddings):")
print(pd.DataFrame(confusion_matrix(y_test, y_pred_emb),
                    index=[f'Actual_{i}' for i in sorted(y.unique())],
                    columns=[f'Predicted_{i}' for i in sorted(y.unique())]))

print("\n--- Script Finished ---")


Evaluating model performance on the Test set (using embeddings)...

Overall Accuracy (Embeddings): 0.6908

Classification Report (Embeddings):
              precision    recall  f1-score   support

           1       0.59      0.71      0.65     13909
           2       0.27      0.26      0.27      5779
           3       0.31      0.32      0.31      7387
           4       0.40      0.26      0.31     11395
           5       0.84      0.85      0.85     60294

    accuracy                           0.69     98764
   macro avg       0.48      0.48      0.48     98764
weighted avg       0.68      0.69      0.68     98764


Confusion Matrix (Embeddings):
          Predicted_1  Predicted_2  Predicted_3  Predicted_4  Predicted_5
Actual_1         9894         1244          877          356         1538
Actual_2         2100         1516          934          322          907
Actual_3         1409          901         2358          819         1900
Actual_4          820          620     

#Concatenating TF-IDF features and averaged embedding features

In [ ]:
# --- Assuming previous steps are done: ---
# - Data loaded into df
# - df cleaned, 'review_cleaned' column exists
# - Data split into X_train, X_test, y_train, y_test
# - FastText model loaded into ft_model, vector_size defined
# - vectorize_text function defined correctly

import numpy as np
from scipy.sparse import hstack # <-- Import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
import time

# --- Step 1: Generate TF-IDF Features ---
# Initialize TF-IDF Vectorizer (Choose your parameters, e.g., max_features)
tfidf_max_features = 10000 # Example value, adjust as needed
print(f"\nInitializing TF-IDF Vectorizer with max_features={tfidf_max_features}...")
tfidf_vectorizer = TfidfVectorizer(max_features=tfidf_max_features)

print("Applying TF-IDF to training data...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
print(f"TF-IDF applied to training data. Shape: {X_train_tfidf.shape}")

print("Applying TF-IDF to testing data...")
X_test_tfidf = tfidf_vectorizer.transform(X_test)
print(f"TF-IDF applied to testing data. Shape: {X_test_tfidf.shape}")


# --- Step 2: Generate Averaged Embedding Features ---
# (Ensure ft_model and vectorize_text are available from previous steps)

if ft_model:
    print("\nVectorizing training data (Averaging FastText Embeddings)...")
    X_train_embeddings = np.array([vectorize_text(text, ft_model, vector_size) for text in tqdm(X_train, desc="Vectorizing Train")])
    print(f"Embedding vectors for training data created. Shape: {X_train_embeddings.shape}")

    print("Vectorizing testing data...")
    X_test_embeddings = np.array([vectorize_text(text, ft_model, vector_size) for text in tqdm(X_test, desc="Vectorizing Test")])
    print(f"Embedding vectors for testing data created. Shape: {X_test_embeddings.shape}")
else:
    print("\nCannot generate embeddings, FastText model not loaded.")
    # Handle error: maybe exit or skip the rest
    X_train_embeddings, X_test_embeddings = None, None


# --- Step 3: Combine Features ---
if X_train_embeddings is not None and X_test_embeddings is not None:
    print("\nCombining TF-IDF and Embedding features...")
    # hstack handles combining sparse (TF-IDF) and dense (Embeddings) arrays
    X_train_combined = hstack((X_train_tfidf, X_train_embeddings))
    X_test_combined = hstack((X_test_tfidf, X_test_embeddings))

    print(f"Combined training features shape: {X_train_combined.shape}")
    print(f"Combined testing features shape: {X_test_combined.shape}")
    # Note: The combined matrix will be sparse
else:
    print("Skipping combination and training as embeddings were not generated.")
    X_train_combined, X_test_combined = None, None

# --- Step 4 & 5: Train and Evaluate on Combined Features ---
if X_train_combined is not None and X_test_combined is not None:
    print("\nTraining Logistic Regression model on COMBINED features...")
    # Initialize Logistic Regression
    log_reg_model_comb = LogisticRegression(
        class_weight='balanced',
        solver='liblinear', # Good for sparse data
        max_iter=2000,
        random_state=42
    )

    start_train = time.time()
    log_reg_model_comb.fit(X_train_combined, y_train) # Fit on combined features
    print(f"Model training completed in {time.time() - start_train:.2f} seconds.")

    print("\nEvaluating model performance on the Test set (using COMBINED features)...")

    # Predict on the combined test set
    y_pred_comb = log_reg_model_comb.predict(X_test_combined)

    # Calculate Accuracy
    accuracy_comb = accuracy_score(y_test, y_pred_comb)
    print(f"\nOverall Accuracy (Combined Features): {accuracy_comb:.4f}")

    # Detailed Classification Report
    print("\nClassification Report (Combined Features):")
    print(classification_report(y_test, y_pred_comb, target_names=[str(i) for i in sorted(y.unique())], zero_division=0))

    # Confusion Matrix
    print("\nConfusion Matrix (Combined Features):")
    print(pd.DataFrame(confusion_matrix(y_test, y_pred_comb),
                       index=[f'Actual_{i}' for i in sorted(y.unique())],
                       columns=[f'Predicted_{i}' for i in sorted(y.unique())]))

print("\n--- Script Finished ---")


Initializing TF-IDF Vectorizer with max_features=10000...
Applying TF-IDF to training data...
TF-IDF applied to training data. Shape: (395056, 10000)
Applying TF-IDF to testing data...
TF-IDF applied to testing data. Shape: (98764, 10000)

Vectorizing training data (Averaging FastText Embeddings)...


Vectorizing Train:   0%|          | 0/395056 [00:00<?, ?it/s]

Embedding vectors for training data created. Shape: (395056, 300)
Vectorizing testing data...


Vectorizing Test:   0%|          | 0/98764 [00:00<?, ?it/s]

Embedding vectors for testing data created. Shape: (98764, 300)

Combining TF-IDF and Embedding features...
Combined training features shape: (395056, 10300)
Combined testing features shape: (98764, 10300)

Training Logistic Regression model on COMBINED features...
Model training completed in 410.33 seconds.

Evaluating model performance on the Test set (using COMBINED features)...

Overall Accuracy (Combined Features): 0.7414

Classification Report (Combined Features):
              precision    recall  f1-score   support

           1       0.69      0.75      0.72     13909
           2       0.32      0.33      0.32      5779
           3       0.38      0.41      0.40      7387
           4       0.48      0.39      0.43     11395
           5       0.88      0.89      0.88     60294

    accuracy                           0.74     98764
   macro avg       0.55      0.55      0.55     98764
weighted avg       0.74      0.74      0.74     98764


Confusion Matrix (Combined Features